<a href="https://colab.research.google.com/github/khalidtouch/electronic-component-classifier/blob/main/cv_part/training_testing_yolov5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch # YOLOv5 implemented using pytorch

In [2]:
from IPython.display import Image #this is to render predictions

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
%cd drive/MyDrive/Development of a Mobile Application and Computer Vision Model for Automated Identification of Electronics Components/Yolo Folder

/content/drive/MyDrive/Development of a Mobile Application and Computer Vision Model for Automated Identification of Electronics Components/Yolo Folder


In [13]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 15598, done.
remote: Counting objects: 100% (205/205), done.
remote: Compressing objects: 100% (151/151), done.
remote: Total 15598 (delta 98), reused 115 (delta 54), pack-reused 15393
Receiving objects: 100% (15598/15598), 14.64 MiB | 5.94 MiB/s, done.
Resolving deltas: 100% (10626/10626), done.


In [14]:
%ls -la

total 4
drwx------ 9 root root 4096 Apr 27 11:14 yolov5/


In [15]:
%cd yolov5

/content/drive/MyDrive/Development of a Mobile Application and Computer Vision Model for Automated Identification of Electronics Components/Yolo Folder/yolov5


In [16]:
#!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 11.3 MB/s eta 0:00:00


In [ ]:
from tqdm import tqdm
import IProgress
from ipywidgets import IntProgress

In [ ]:
#torch.cuda.get_arch_list() 

In [ ]:
print('All set. Using PyTorch version %s with %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

All set. Using PyTorch version 1.8.1+cu111 with _CudaDeviceProperties(name='GeForce RTX 3090', major=8, minor=6, total_memory=24576MB, multi_processor_count=82)


# Training the yolov5 Model

 Once all the files are cloned, you’ll need to move the dataset’s dataset.yaml file to the yolov5/data directory. 
 This file contains information required by YOLO to train the model on the custom data. 

In [ ]:
!python train.py --img 416 --batch 16 --epochs 50 --data dataset.yaml --weights yolov5s.pt --cache

At the end of the training, two files should be saved in yolov5/runs/train/exp/weights: last.pt and best.pt. We’ll use best.pt.

If you want to explore the metrics recorded during training, I suggest you use TensorBoard, a very interactive exploration tool:

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

# Testing the Model on Google Colab
Let’s explore now how confident our model is. We can plot a validation batch obtained during training and inspect the confidence score of each label:

In [ ]:
Image(filename='runs/train/exp2/val_batch0_labels.jpg', width=1000)

# Testing the Model:

In [ ]:
!python detect.py --weights runs/train/exp2/weights/best.pt --img 416 --conf 0.25 --source runs/train/exp2/test_images

# Yolov5 model is trained and tested. 

# Now next step is to convert it into a tflite model, so that we can use it on android phone


### you have best.pt and last.pt inside runs\train\exp2\weights

In [ ]:
# Convert weights to fp16 TFLite model

!python export.py --weights runs/train/exp2/weights/best.pt --include tflite --img 416

In [ ]:
#After running the above command, you will have best-fp16.tflite in runs\train\exp2\weights folder

# Lets run the created tflite model.

!python detect.py --weights runs\train\exp2\weights\best-fp16.tflite --img 416 --conf 0.25 --source runs/train/exp2/test_images

# Our model is working fine and results are stored in runs\detect\exp3

In [ ]:
#Convert weights to int8 TFLite model

#!python export.py --weights runs/train/exp2/weights/best.pt --include tflite --int8 --img 416 --data data/coco128.yaml

## 3. Get the android folder from here
https://github.com/AarohiSingla/TFLite-Object-Detection-Android-App-Tutorial-Using-YOLOv5

### Now Put TFLite models in assets folder of Android project, and change

And then open android/app/src/main/java/org/tensorflow/lite/examples/detection/tflite/DetectorFactory.java

Do the following changes:

Change the inputSize of image as per your dataset.

Change labelFilename according to the classes of the model

    
Then run the program in Android Studio.